# Notebook description

This notebook contains the code used for generating different tables used by the rest of the noteeboks. In most cases, such tables are just samples of the original tables in the Challenge dataset, but some of them are bit more involved. Using these generated tables allows for lower running times in the rest of the notebooks. 

Having this code in a separate notebook allows for cleaner notebooks.

# Imports and main params

In [1]:
import numpy as np
import pandas as pd

In [2]:
random_state = 0

# Useful functions

In [7]:
def extract_json_to_csv(json_path, csv_path, keep_cols = None, smp_frac = None, 
                        chunksize = 100000, encoding = None, random_state = None):
    """ Reads a json file and saves it as csv, optionally filtering rows and columns. 
        
    Parameters
    ----------
    json_path : string
        path to the json file
    csv_path : string
        path to the csv file
    keep_cols : list of strings, optional
        column names of the columns to keep
    smp_frac : float, optional
        fraction of rows from the json file to keep
    chunksize : int, optional
        number of rows to read from the json file per iteration
    encoding: string, optional
        econding to use in the csv file
    random_state: int
        seed for sampling the json file 
    """
    reader = pd.read_json(json_path, lines=True, chunksize=chunksize)
    df = pd.DataFrame()
    for chunk in reader:
        if keep_cols is not None:
            chunk = chunk[keep_cols]
        if smp_frac is not None:
            chunk = chunk.sample(frac=smp_frac, random_state=random_state)
        print('Concatenating new chunk...')
        df = pd.concat([df, chunk])
        print('Read {} rows'.format(df.shape[0]))
        
    print('Saving CSV...')
    df.to_csv(csv_path, index=False, encoding=encoding)

# Dataset generation

## Reviews

In [8]:
reviews_path = '../data/yelp_academic_dataset_review.json'
generated_reviews_path = '../data/reviews.csv'
keep_cols = ['business_id', 'date', 'stars', 'user_id'] 
extract_json_to_csv(reviews_path, generated_reviews_path, keep_cols, chunksize = 500000)

Concatenating new chunk...
Read 500000 rows
Concatenating new chunk...
Read 1000000 rows
Concatenating new chunk...
Read 1500000 rows
Concatenating new chunk...
Read 2000000 rows
Concatenating new chunk...
Read 2500000 rows
Concatenating new chunk...
Read 3000000 rows
Concatenating new chunk...
Read 3500000 rows
Concatenating new chunk...
Read 4000000 rows
Concatenating new chunk...
Read 4500000 rows
Concatenating new chunk...
Read 5000000 rows
Concatenating new chunk...
Read 5500000 rows
Concatenating new chunk...
Read 5996996 rows
Saving CSV...


## Users

In [10]:
users_path = '../data/yelp_academic_dataset_user.json'
generated_users_path = '../data/users.csv'
keep_cols = ['user_id', 'friends'] 
extract_json_to_csv(users_path, generated_users_path, keep_cols, smp_frac = 0.1,
                    chunksize = 100000, encoding='utf-8', random_state=random_state)

Concatenating new chunk...
Read 10000 rows
Concatenating new chunk...
Read 20000 rows
Concatenating new chunk...
Read 30000 rows
Concatenating new chunk...
Read 40000 rows
Concatenating new chunk...
Read 50000 rows
Concatenating new chunk...
Read 60000 rows
Concatenating new chunk...
Read 70000 rows
Concatenating new chunk...
Read 80000 rows
Concatenating new chunk...
Read 90000 rows
Concatenating new chunk...
Read 100000 rows
Concatenating new chunk...
Read 110000 rows
Concatenating new chunk...
Read 120000 rows
Concatenating new chunk...
Read 130000 rows
Concatenating new chunk...
Read 140000 rows
Concatenating new chunk...
Read 150000 rows
Concatenating new chunk...
Read 151817 rows
Saving CSV...


## Businesses

In [11]:
businesses_path = '../data/yelp_academic_dataset_business.json'
generated_businesses_path = '../data/businesses.csv'
extract_json_to_csv(businesses_path, generated_businesses_path, chunksize = 500000, encoding='utf-8')

Concatenating new chunk...
Read 188593 rows
Saving CSV...


## Checkins

The following cell creates a checkings CSV with just two columns (apart from business_id), corresponding to the number of checkins on weekends and on working days.

In [12]:
checkins_path = '../data/yelp_academic_dataset_checkin.json'
generated_checkins_path = '../data/checkins.csv'


reader = pd.read_json(checkins_path, lines=True, chunksize=20000)
df = pd.DataFrame()
for chunk in reader:
    print('Processing chunk')
    chunk = pd.concat([chunk.drop(['time'], axis=1), chunk['time'].apply(pd.Series)], axis=1).fillna(0)
    weekend_cols = [col for col in chunk.drop('business_id', axis=1).columns if \
                    (col.split('-')[0] in ['Sat', 'Sun'])]
    week_cols = [col for col in chunk.drop('business_id', axis=1).columns if \
                 (col.split('-')[0] in ['Mon', 'Tue', 'Wed', 'Thu', 'Fri'])]
    chunk['weekends'] = chunk[weekend_cols].sum(axis=1)
    chunk['week'] = chunk[week_cols].sum(axis=1)
    chunk = chunk[['business_id', 'weekends', 'week']]
    print('Concatenating new chunk...')
    df = pd.concat([df, chunk])
    print('Read {} rows'.format(df.shape[0]))
df.to_csv(generated_checkins_path, index=False, encoding='utf-8')

Processing chunk
Concatenating new chunk...
Read 20000 rows
Processing chunk
Concatenating new chunk...
Read 40000 rows
Processing chunk
Concatenating new chunk...
Read 60000 rows
Processing chunk
Concatenating new chunk...
Read 80000 rows
Processing chunk
Concatenating new chunk...
Read 100000 rows
Processing chunk
Concatenating new chunk...
Read 120000 rows
Processing chunk
Concatenating new chunk...
Read 140000 rows
Processing chunk
Concatenating new chunk...
Read 157075 rows


## Friends

The 3 cells below create a handy mapping between friends. The generated table is based on a sample from the generated users table (run the cell under _Users_ first if you are going to run the cells below). First, users with less than a certain number of friends are filtered out. Then, a number of users are randomly sampled. The final table contains two columns: 'user_id' and 'friend_id'. A row [user_id, friend_id] is present in the final table if and only if user_id belongs to the sampled table and user_id has friend_id among her friends. 

Params:

In [3]:
generated_users_path = '../data/users.csv'
friends_path = '../data/friends.csv'

# Min number of friends. Users with less friends than this value are filtered out
min_n_friends = 20

Read users, preprocess friends column and filter out users with less than min_n_friends friends

In [4]:
users = pd.read_csv(generated_users_path)
users = users[users.friends != 'None'].copy()
users['friends'] = users.friends.apply(lambda x: [i.strip() for i in x.split(',')])
users = users[users.friends.apply(lambda x: len(x) >= 20)].copy()

Explode list of friends and save

In [5]:
# Efficent function for exploding a list column, creating a new row for each element in list
# Source: https://github.com/pandas-dev/pandas/issues/10511
def unlistify(df, column):
    matches = [i for i,n in enumerate(df.columns) if n==column]

    if len(matches)==0:
        raise Exception('Failed to find column named ' + column +'!')
    if len(matches)>1:
        raise Exception('More than one column named ' + column +'!')

    col_idx = matches[0]
    
    # Helper function to expand and repeat the column col_idx
    def fnc(d):
        row = list(d.values[0])
        bef = row[:col_idx]
        aft = row[col_idx+1:]
        col = row[col_idx]
        z = [bef + [c] + aft for c in col]
        return pd.DataFrame(z)
    
    col_idx += len(df.index.shape) # Since we will push reset the index
    index_names = list(df.index.names)
    column_names = list(index_names) + list(df.columns)
    return (df
            .reset_index()
            .groupby(level=0,as_index=0)
            .apply(fnc)
            .rename(columns = lambda i :column_names[i])
            .set_index(index_names)
           )

In [6]:
users = unlistify(users, 'friends')
users.columns = ['user_id', 'friend_id']
users.to_csv(friends_path, index=False)

In [ ]:
# friends_path = '../data/friends.csv'
# users = users.friends.apply(pd.Series) \
#              .stack() \
#              .reset_index(level=1, drop=True) \
#              .to_frame('friend_id') \
#              .join(users.drop('friends', axis=1)) \
#              .reset_index(level=0, drop=True)
# users = users[users.friend_id != '']
# users.to_csv(friends_path, index=False)